In [3]:
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bson.json_util import dumps
import base64

# Import the animal shelter class from the CRUD module
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Initialize the Animal Shelter with credentials
username = "john"
password = "john"
shelter = AnimalShelter(username, password)

# Load initial data
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the _id column to prevent ObjectID-related crashes
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Load and encode the logo image
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Hidden div for storing data
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Header with title and branding
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Created by Your Name'))),
    
    # Logo with link
    html.Center(
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'height': '100px'}
            ),
            href='https://www.snhu.edu'
        )
    ),
    
    html.Hr(),
    
    # Radio Items for filtering
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            style={'margin': '10px'}
        )
    ]),
    
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    
    html.Br(),
    html.Hr(),
    
    # Charts container
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 # Breed Chart
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 # Map
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'water':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    elif filter_type == 'mount':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                             "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                             "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    
    # Remove _id if present
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    return df.to_dict('records'), columns

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    breed_counts = dff['breed'].value_counts()
    
    return [
        dcc.Graph(
            figure=px.pie(
                values=breed_counts.values,
                names=breed_counts.index,
                title='Dog Breed Distribution'
            )
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if not viewData or not selected_rows:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    row = dff.iloc[selected_rows[0]]
    
    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[row['location_lat'], row['location_long']],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[row['location_lat'], row['location_long']],
                    children=[
                        dl.Tooltip(row['breed']),
                        dl.Popup([
                            html.H3("Animal Details"),
                            html.P(f"Name: {row['name']}"),
                            html.P(f"Breed: {row['breed']}"),
                            html.P(f"Age: {row['age_upon_outcome']}")
                        ])
                    ]
                )
            ]
        )
    ]

if __name__ == '__main__':
    app.run_server(debug=True)

ERROR:root:Failed to connect to MongoDB
ERROR:root:localhost:39329: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 675f5bf8dc20c03f22a17451, topology_type: Single, servers: [<ServerDescription ('localhost', 39329) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:39329: [Errno 111] Connection refused')>]>


ServerSelectionTimeoutError: localhost:39329: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 675f5bf8dc20c03f22a17451, topology_type: Single, servers: [<ServerDescription ('localhost', 39329) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:39329: [Errno 111] Connection refused')>]>